## 3. 모델링
XG부스트 모델, 나머지 두개는 딥러닝 기반의 모델로서 하나는 합성곱 신경망을 활용한 모델과 맨하탄 거리르 호라용하는 LSTM 모델인 MaLSTM 모델.

### XG 부스트 텍스트 유사도 분석 모델
맨 먼저 사용할 모델은 앙상블 모델 중 하나인 XG 부스트 모델.  
해당 모델을 사용해서 데이터의 주어진 두 질문 문장 사이의 유사도를 측정해서 두 질문이 중복인지 아닌지를 판단.  
우선 XG 부스트 모델이 어떤 모델인지 먼저 알아보자. 

#### 모델 소개
**XG 부스트** : eXtream Gradient Boosting의 약자로 최근 캐글 사용자에게 큰 인기를 얻고 있는 모델 중 하나.  
앙상블의 한 방법인 부스팅(Boosting) 기법을 사용하는 방법. 

#### 모델 구현

In [16]:
import pandas as pd

In [2]:
import numpy as np

DATA_IN_PATH = './data_in/'

TRAIN_Q1_DATA_FILE = 'q1_train.npy'
TRAIN_Q2_DATA_FILE = 'q2_train.npy'
TRAIN_LABEL_DATA_FILE = 'label_train.npy'

# 훈련 데이터 가져오기
train_q1_data = np.load(open(DATA_IN_PATH + TRAIN_Q1_DATA_FILE, 'rb'))
train_q2_data = np.load(open(DATA_IN_PATH + TRAIN_Q2_DATA_FILE, 'rb'))
train_labels = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA_FILE, 'rb'))

현재 두 질문이 따로 구성돼 있는데 이를 하나씩 묶어 하나의 질문 쌍으로 만들자.

In [4]:
train_input = np.stack((train_q1_data, train_q2_data), axis = 1)


stack 함수를 통해 두 질문을 하나의 쌍으로 만든다.  
예를 들어 질문[A]와 질문 [B]가 있을 때 이 질문을 하나로 묶어 [[A], [B]]로 만드는 것이다.

In [6]:
print(train_input.shape)

(298526, 2, 31)


이제 학습 데이터의 일부를 모델 검증을 위한 검증 데이터로 만들어 두자

In [7]:
from sklearn.model_selection import train_test_split

train_input, eval_input, train_label, eval_label = train_test_split(train_input, train_labels, test_size = 0.2, random_state = 4242)

모델을 구현하기 위해서 **xgboost** 라는 라이브러리를 활용할 것이다.  
우선 해당 라이브러리가 설치돼 잇지 않다면 설치하자.

`https://xgboost.readthedocs.io/en/latest/`



In [8]:
import xgboost as xgb

In [9]:
train_data = xgb.DMatrix(train_input.sum(axis = 1), label = train_label)
eval_data = xgb.DMatrix(eval_input.sum(axis = 1), label = eval_label)

In [10]:
data_list = [(train_data, 'train'), (eval_data, 'valid')]

입력값을 xgb 라이브러리의 데이터 형식은 DMatrix 형태로 만들어야 함.  
학습 데이터와 검증 데이터 모두 적용해서 해당 데이터 형식으로 만든다.  

적용 과정에서 각 데이터에 대해 sum 함수를 사용하는데 이는 각 데이터의 두 질문을 하나의 값으로 만들어 주기 위해서임.  
그리고 두 개의 데이터를 묶어서 하나의 리스트로 만든다.  
이때 학습 데이터와 검증 데이터는 각 상태의 문자열과 함께 튜플 형태로 구성!

이제 모델을 생성하고 학습하는 과정을 진행해 보자.

In [12]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'rmse'

bst = xgb.train(params, train_data, num_boost_round = 1000, evals = data_list, early_stopping_rounds = 10)

[0]	train-rmse:0.48373	valid-rmse:0.48421
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[1]	train-rmse:0.47345	valid-rmse:0.47447
[2]	train-rmse:0.46689	valid-rmse:0.46833
[3]	train-rmse:0.46209	valid-rmse:0.46363
[4]	train-rmse:0.45849	valid-rmse:0.46035
[5]	train-rmse:0.45557	valid-rmse:0.45770
[6]	train-rmse:0.45338	valid-rmse:0.45578
[7]	train-rmse:0.45078	valid-rmse:0.45341
[8]	train-rmse:0.44942	valid-rmse:0.45226
[9]	train-rmse:0.44825	valid-rmse:0.45136
[10]	train-rmse:0.44705	valid-rmse:0.45041
[11]	train-rmse:0.44494	valid-rmse:0.44856
[12]	train-rmse:0.44417	valid-rmse:0.44795
[13]	train-rmse:0.44357	valid-rmse:0.44741
[14]	train-rmse:0.44234	valid-rmse:0.44624
[15]	train-rmse:0.44168	valid-rmse:0.44565
[16]	train-rmse:0.44113	valid-rmse:0.44528
[17]	train-rmse:0.44064	valid-rmse:0.44494
[18]	train-rmse:0.43962	valid-rmse:0.44406
[19]	train-rmse:0.43930	valid-rmse:0.44386
[20]	

[186]	train-rmse:0.40121	valid-rmse:0.42414
[187]	train-rmse:0.40099	valid-rmse:0.42406
[188]	train-rmse:0.40080	valid-rmse:0.42405
[189]	train-rmse:0.40079	valid-rmse:0.42405
[190]	train-rmse:0.40077	valid-rmse:0.42404
[191]	train-rmse:0.40056	valid-rmse:0.42398
[192]	train-rmse:0.40037	valid-rmse:0.42395
[193]	train-rmse:0.40020	valid-rmse:0.42390
[194]	train-rmse:0.40005	valid-rmse:0.42386
[195]	train-rmse:0.39995	valid-rmse:0.42384
[196]	train-rmse:0.39993	valid-rmse:0.42383
[197]	train-rmse:0.39992	valid-rmse:0.42383
[198]	train-rmse:0.39991	valid-rmse:0.42382
[199]	train-rmse:0.39975	valid-rmse:0.42379
[200]	train-rmse:0.39959	valid-rmse:0.42373
[201]	train-rmse:0.39958	valid-rmse:0.42372
[202]	train-rmse:0.39957	valid-rmse:0.42372
[203]	train-rmse:0.39953	valid-rmse:0.42371
[204]	train-rmse:0.39951	valid-rmse:0.42372
[205]	train-rmse:0.39918	valid-rmse:0.42351
[206]	train-rmse:0.39897	valid-rmse:0.42349
[207]	train-rmse:0.39865	valid-rmse:0.42338
[208]	train-rmse:0.39843	valid-r

[373]	train-rmse:0.37854	valid-rmse:0.41969
[374]	train-rmse:0.37853	valid-rmse:0.41968
[375]	train-rmse:0.37831	valid-rmse:0.41966
[376]	train-rmse:0.37827	valid-rmse:0.41965
[377]	train-rmse:0.37814	valid-rmse:0.41962
[378]	train-rmse:0.37801	valid-rmse:0.41957
[379]	train-rmse:0.37787	valid-rmse:0.41957
[380]	train-rmse:0.37768	valid-rmse:0.41957
[381]	train-rmse:0.37753	valid-rmse:0.41956
[382]	train-rmse:0.37749	valid-rmse:0.41954
[383]	train-rmse:0.37745	valid-rmse:0.41954
[384]	train-rmse:0.37727	valid-rmse:0.41954
[385]	train-rmse:0.37718	valid-rmse:0.41952
[386]	train-rmse:0.37717	valid-rmse:0.41951
[387]	train-rmse:0.37716	valid-rmse:0.41951
[388]	train-rmse:0.37706	valid-rmse:0.41950
[389]	train-rmse:0.37704	valid-rmse:0.41950
[390]	train-rmse:0.37693	valid-rmse:0.41947
[391]	train-rmse:0.37670	valid-rmse:0.41941
[392]	train-rmse:0.37661	valid-rmse:0.41938
[393]	train-rmse:0.37654	valid-rmse:0.41938
[394]	train-rmse:0.37652	valid-rmse:0.41937
[395]	train-rmse:0.37652	valid-r

[560]	train-rmse:0.35991	valid-rmse:0.41676
[561]	train-rmse:0.35972	valid-rmse:0.41674
[562]	train-rmse:0.35962	valid-rmse:0.41675
[563]	train-rmse:0.35940	valid-rmse:0.41666
[564]	train-rmse:0.35927	valid-rmse:0.41663
[565]	train-rmse:0.35919	valid-rmse:0.41663
[566]	train-rmse:0.35899	valid-rmse:0.41664
[567]	train-rmse:0.35891	valid-rmse:0.41664
[568]	train-rmse:0.35883	valid-rmse:0.41662
[569]	train-rmse:0.35876	valid-rmse:0.41661
[570]	train-rmse:0.35858	valid-rmse:0.41664
[571]	train-rmse:0.35847	valid-rmse:0.41662
[572]	train-rmse:0.35837	valid-rmse:0.41662
[573]	train-rmse:0.35819	valid-rmse:0.41657
[574]	train-rmse:0.35800	valid-rmse:0.41657
[575]	train-rmse:0.35791	valid-rmse:0.41657
[576]	train-rmse:0.35787	valid-rmse:0.41658
[577]	train-rmse:0.35785	valid-rmse:0.41658
[578]	train-rmse:0.35770	valid-rmse:0.41659
[579]	train-rmse:0.35762	valid-rmse:0.41659
[580]	train-rmse:0.35761	valid-rmse:0.41659
[581]	train-rmse:0.35738	valid-rmse:0.41656
[582]	train-rmse:0.35729	valid-r

[747]	train-rmse:0.34239	valid-rmse:0.41509
[748]	train-rmse:0.34230	valid-rmse:0.41506
[749]	train-rmse:0.34227	valid-rmse:0.41507
[750]	train-rmse:0.34223	valid-rmse:0.41508
[751]	train-rmse:0.34221	valid-rmse:0.41508
[752]	train-rmse:0.34214	valid-rmse:0.41506
[753]	train-rmse:0.34212	valid-rmse:0.41505
[754]	train-rmse:0.34195	valid-rmse:0.41499
[755]	train-rmse:0.34187	valid-rmse:0.41500
[756]	train-rmse:0.34180	valid-rmse:0.41499
[757]	train-rmse:0.34163	valid-rmse:0.41497
[758]	train-rmse:0.34147	valid-rmse:0.41495
[759]	train-rmse:0.34128	valid-rmse:0.41492
[760]	train-rmse:0.34114	valid-rmse:0.41490
[761]	train-rmse:0.34104	valid-rmse:0.41489
[762]	train-rmse:0.34101	valid-rmse:0.41490
[763]	train-rmse:0.34098	valid-rmse:0.41487
[764]	train-rmse:0.34083	valid-rmse:0.41485
[765]	train-rmse:0.34066	valid-rmse:0.41483
[766]	train-rmse:0.34053	valid-rmse:0.41477
[767]	train-rmse:0.34044	valid-rmse:0.41475
[768]	train-rmse:0.34039	valid-rmse:0.41476
[769]	train-rmse:0.34031	valid-r

[934]	train-rmse:0.32754	valid-rmse:0.41364
[935]	train-rmse:0.32748	valid-rmse:0.41363
[936]	train-rmse:0.32741	valid-rmse:0.41365
[937]	train-rmse:0.32732	valid-rmse:0.41369
[938]	train-rmse:0.32715	valid-rmse:0.41369
[939]	train-rmse:0.32703	valid-rmse:0.41370
[940]	train-rmse:0.32694	valid-rmse:0.41371
[941]	train-rmse:0.32682	valid-rmse:0.41367
[942]	train-rmse:0.32678	valid-rmse:0.41368
[943]	train-rmse:0.32669	valid-rmse:0.41368
[944]	train-rmse:0.32656	valid-rmse:0.41371
[945]	train-rmse:0.32646	valid-rmse:0.41370
Stopping. Best iteration:
[935]	train-rmse:0.32748	valid-rmse:0.41363



우선 모델을 만들고 학습하기 위해 몇 가지 선택해야 하는 옵션은 **딕셔너리를 만들어 넣으면 됨**  
이때 이 딕셔너리에는 모델의 목적함수와 평가 지표를 정해서 넣어야 함.  
여기서는 우선 목적함수의 경우 **이진 로지스틱 함수** 사용  
평가 지표의 경우 **rmse(Root mean squared error)** 사용  
이렇게 만든 인자와 학습 데이터, 데이터를 반복하는 횟수인 num_boost_round, 모델 검증 시 사용할 전체 데이터 쌍, 그리고 early stopping을 위한 횟수를 지정.  

데이터 반복 횟수, 즉 epoch을 의미하는 값으로는 1000 설정.  
이는 전체 데이터를 만번 반복해야 끝나도록 설정한 것.  
또한 early stopping을 위한 횟수값으로 10을 설정했는데 이는 만약 10 epoch 동안 에러값이 별로 줄어들지 않았을 경우에는 학습을 early stopping하게 하는 것.

위에 학습 과정을 보면 각 step마다 학습 에러와 검증 에러를 계속 보여주고 있으며, 945 스텝에서 학습이 끝났다.  

이제 만든 모델을 사용해 평가 데이터를 예측하고 예측 결과를 kaggle에 제출할 수 있게 파일로 만들어보자.

In [13]:
TEST_Q1_DATA_FILE = 'test_q1.npy'
TEST_Q2_DATA_FILE = 'test_q2.npy'
TEST_ID_DATA_FILE = 'test_id.npy'

test_q1_data = np.load(open(DATA_IN_PATH + TEST_Q1_DATA_FILE, 'rb'))
test_q2_data = np.load(open(DATA_IN_PATH + TEST_Q2_DATA_FILE, 'rb'))
#test_id_data = np.load(open(DATA_IN_PATH + TEST_ID_DATA_FILE, 'rb'))

In [22]:
test_data = pd.read_csv(DATA_IN_PATH + 'test.csv', encoding = "utf-8")
valid_ids = [type(x) == int for x in test_data.test_id]
test_data = test_data[valid_ids].drop_duplicates()

C:\Users\nyong\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
test_id = np.array(test_data['test_id'])

print('Shape of questions1 data : {}'.format(test_q1_data.shape))
print("Shape of questions2 data : {}".format(test_q2_data.shape))
print('Shape of ids : {}'.format(test_id.shape))

Shape of questions1 data : (2345796, 31)
Shape of questions2 data : (2345796, 31)
Shape of ids : (2345796,)


In [24]:
test_input = np.stack((test_q1_data, test_q2_data), axis = 1)
test_data = xgb.DMatrix(test_input.sum(axis = 1))

test_predict = bst.predict(test_data)

In [25]:
import os
DATA_OUT_PATH = './data_out/'

if not os.path.exists(DATA_OUT_PATH) :
    os.makedirs(DATA_OUT_PATH)
    
output = pd.DataFrame({'test_id': test_id, 'is_duplicate': test_predict})
output.to_csv(DATA_OUT_PATH + 'simple_xgb.csv', index = False)